In [4]:
!uv pip install langchain langchain-google-genai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.12.1 environment at: /workspaces/End2End_RAG/.venv
Audited 6 packages in 12ms


In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

## Data Ingestion

In [8]:
from langchain_community.document_loaders import TextLoader

In [9]:
loader = TextLoader("/workspaces/End2End_RAG/data/Way of All Earth.txt", encoding="utf8")
document = loader.load()

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_splitter.split_documents(document)

In [14]:
!uv pip install sentence-transformers

Using Python 3.12.1 environment at: /workspaces/End2End_RAG/.venv


Resolved 47 packages in 1.64s                                        
⠙ Preparing packages... (0/33)                                                  
⠙ Preparing packages... (0/33)------------------     0 B/3.17 MiB            
⠙ Preparing packages... (0/33)------------------     0 B/3.17 MiB            
fsspec               ------------------------------     0 B/196.26 KiB
⠙ Preparing packages... (0/33)------------------     0 B/3.17 MiB            
fsspec               ------------------------------     0 B/196.26 KiB
⠙ Preparing packages... (0/33)------------------     0 B/3.17 MiB            
fsspec               ------------------------------     0 B/196.26 KiB
⠙ Preparing packages... (0/33)------------------     0 B/3.17 MiB            
markupsafe           ------------------------------     0 B/22.41 KiB
fsspec               ------------------------------     0 B/196.26 KiB
⠙ Preparing packages... (0/33)------------------     0 B/3.17 MiB            
markupsafe           ------

In [15]:
!uv pip install langchain-huggingface

Using Python 3.12.1 environment at: /workspaces/End2End_RAG/.venv
Resolved 32 packages in 2.33s                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------------     0 B/29.22 KiB          
⠙ Preparing packages... (0/1)m--------------- 14.87 KiB/29.22 KiB        
⠙ Preparing packages... (0/1)----------- 29.22 KiB/29.22 KiB        
Prepared 1 package in 78ms                                                        
░░░░░░░░░░░░░░░░░░░░ [0/1] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 6msce==1.1.0                         
 + langchain

In [19]:
!uv pip install faiss-cpu

Using Python 3.12.1 environment at: /workspaces/End2End_RAG/.venv
⠙ Resolving dependencies...                                                     

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Resolved 3 packages in 658ms                                         
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)-------------------     0 B/22.50 MiB           
⠙ Preparing packages... (0/1)------------------- 14.91 KiB/22.50 MiB         
⠙ Preparing packages... (0/1)------------------- 30.91 KiB/22.50 MiB         
⠙ Preparing packages... (0/1)------------------- 42.35 KiB/22.50 MiB         
⠙ Preparing packages... (0/1)------------------- 47.83 KiB/22.50 MiB         
⠙ Preparing packages... (0/1)------------------- 53.32 KiB/22.50 MiB         
⠙ Preparing packages... (0/1)------------------- 58.80 KiB/22.50 MiB         
⠙ Preparing packages... (0/1)------------------- 64.29 KiB/22.50 MiB         
⠙ Preparing packages... (0/1)------------------- 75.26 KiB/22.50 MiB         
⠙ Preparing packages... (0/1)------------------- 78.91 KiB/22.50 MiB         
⠙ Preparing packages... (0/1)------------------- 94.91 KiB/22.50 MiB 

In [16]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()

In [17]:
from langchain_community.vectorstores import FAISS

In [20]:
vectorstore = FAISS.from_documents(chunks, embedding)

In [21]:
template = """
You are a very useful Q&A assistant that makes use of the context and answer the question.
If you don't know the answer just say 'I do not know'. Keep the answer concise.
Question: {question}
Context: {context}
"""